In [1]:
import pandas as pd
import numpy as np
import os

DATA_PATH = '/kaggle/input/kdtai-1/COVID_19_XRAY'

TRAIN_PATH = os.path.join(DATA_PATH, "train")
TEST_PATH = os.path.join(DATA_PATH, "test")
TRAIN_LABEL_PATH = os.path.join(TRAIN_PATH, 'labels.csv')

labels_df = pd.read_csv(TRAIN_LABEL_PATH)
labels_df.head(5)


,filename,label
0,image_001.png,covid
1,image_002.png,covid
2,image_003.png,normal
3,image_004.png,normal
4,image_005.png,normal


In [2]:
import shutil

label_list = [ "covid", "normal" ]
_DATA_PATH = "/kaggle/working"

_TRAIN_PATH = os.path.join(_DATA_PATH, "train")
_TEST_PATH = os.path.join(_DATA_PATH, "test")
TRAIN_LABEL_PATH = os.path.join(TRAIN_PATH, 'labels.csv')


for label in label_list :
  TARGET_PATH = os.path.join(_TRAIN_PATH, label)

  os.makedirs(TARGET_PATH, exist_ok = True)
  file_list = labels_df[labels_df["label"] == label]["filename"].tolist()

  for filename in file_list :
    src = os.path.join(TRAIN_PATH, filename)
    dst = os.path.join(TARGET_PATH, filename)

    shutil.copy(src, dst)


In [3]:
NEW_TEST_PATH = os.path.join(_TEST_PATH, "test")
test_file_list = os.listdir(TEST_PATH)

os.makedirs(NEW_TEST_PATH, exist_ok = True)
for filename in test_file_list :
  src = os.path.join(TEST_PATH, filename)
  dst = os.path.join(NEW_TEST_PATH, filename)

  shutil.copy(src, dst)

In [4]:
import numpy as np
import random
import os


import torchvision
from torchvision import transforms 

from torch.utils.data import DataLoader, random_split
from torch import nn
import torch

import timm
from tqdm import tqdm

In [5]:
class effinet_test(nn.Module) :
    def __init__(self, n_outputs:int, **kwargs):
        super(effinet_test, self).__init__()
        self.model = timm.create_model('efficientnetv2_rw_m', pretrained=True, num_classes=0)
        self.classifier = nn.Sequential(
            nn.Linear(in_features = 2152, out_features=512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=512, out_features=n_outputs)
        )

    def forward(self, x):
        output = self.model(x)
        output = self.classifier(output)
        
        return output

이제 이를 모듈화시켜서 앙상블 학습을 진행해보자!

In [6]:
class Full_Model() :
  def __init__(self, seed, save_path, epoch, max_patience) :
    self.save_path = save_path
    self.epoch = epoch
    self.max_patience = max_patience

    self._seed_fix(seed)
    self._get_dataloader()
    self._set_model_env()


  def _seed_fix(self, seed) :
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)

    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False

    self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


  def _get_dataloader(self) :
    train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5), 
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.465, 0.406),
                         (0.229, 0.224, 0.225))
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.465, 0.406),
                            (0.229, 0.224, 0.225))
        
    ])


    TOT_DSET = torchvision.datasets.ImageFolder(root = _TRAIN_PATH, transform = train_transform)

    dataset_size = len(TOT_DSET)
    train_size = int(dataset_size * 0.9)
    validation_size = int(dataset_size * 0.1)

    train_dset, val_dset = random_split(TOT_DSET, [train_size, validation_size])
    test_dset = torchvision.datasets.ImageFolder(root = _TEST_PATH, transform = test_transform)

    self.train_dataloader = DataLoader(dataset = train_dset, batch_size = 8, shuffle = True)
    self.val_dataloader = DataLoader(dataset = val_dset, batch_size = 8, shuffle = True)
    self.test_dataloader = DataLoader(dataset = test_dset, batch_size = 8)

  def _set_model_env(self) :
    self.model = effinet_test(n_outputs = 2)
    self.model.to(self.device)
    self.criterion = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
    self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.8)

  def _train_one_epoch(self) :
    self.train_acc = 0.
    self.train_loss = 0.
    self.model.train()

    tot_size = 0.
    for img, label in tqdm(self.train_dataloader) :
      img = img.to(self.device)
      label = label.to(self.device)

      self.optimizer.zero_grad()
      output = self.model(img)

      loss = self.criterion(output, label)

      _, preds = torch.max(output.data, 1)
      self.train_loss += loss.item()
      self.train_acc += torch.sum( preds == label.data ).item()
      tot_size += label.size(0)

      loss.backward()
      self.optimizer.step()

    self.scheduler.step()

    self.train_acc /= tot_size
    self.train_loss /= tot_size

  def _val_one_epoch(self) :
    self.val_acc = 0.
    self.val_loss = 0.

    self.model.eval()
    tot_size = 0.

    with torch.no_grad() :
      for img, label in tqdm(self.val_dataloader) :
        img = img.to(self.device)
        label = label.to(self.device)

        output = self.model(img)
        loss = self.criterion(output, label)

        _, preds = torch.max(output.data, 1)
        self.val_loss += loss.item()
        self.val_acc += torch.sum( preds == label.data ).item()
        tot_size += label.size(0)

    self.val_acc /= tot_size
    self.val_loss /= tot_size

  def train(self) :
    self.max_val_acc = 0.
    self.min_val_loss = float('inf')
    patience = 0.
    for i in range(self.epoch) :
      self._train_one_epoch()
      self._val_one_epoch()
      
      print( " [ epoch : {:03d} ] train_acc : {:0.03f}, train_loss : {:0.03f}, val_acc : {:0.03f}, val_loss : {:0.03f} ".format(i+1, self.train_acc, self.train_loss, self.val_acc, self.val_loss))
      

      if self.max_val_acc < self.val_acc or self.max_val_acc == self.val_acc and self.min_val_loss > self.val_loss :
        print("Deteced New Best Acc, Model Save....")
        torch.save(self.model, self.save_path)
        patience = 0
        self.max_val_acc = self.val_acc
        self.min_val_loss = self.val_loss

      else :
        patience += 1

        if patience > self.max_patience :
          print("early stopping")
          return

  def predict(self) :
    model = torch.load(self.save_path)
    model.to(self.device)
    self.pred_list = np.array([])

    with torch.no_grad() :
      for img, _ in tqdm(self.test_dataloader) :
        img = img.to(self.device)

        output = model(img)
        _, preds = torch.max(output.data, 1)

        self.pred_list = np.append(self.pred_list, preds.cpu().numpy(), axis = 0)

  def predict_soft(self) :
    model = torch.load(self.save_path)
    model.to(self.device)
    self.pred_list = list()
    
    with torch.no_grad() :
      for img, _ in tqdm(self.test_dataloader) :
        img = img.to(self.device)

        output = model(img)

        self.pred_list.append(output.cpu().numpy())
    self.pred_list = np.concatenate(self.pred_list, axis = 0)
        
  def get_predict_list(self) :
    return self.pred_list

In [7]:

full_pred_list = None

threshold = 0.98
max_idx = 5
idx = 1000

while True :
  SAVE_PATH = os.path.join(_TRAIN_PATH, "best_model_{:d}.h5".format(idx))
  full_model = Full_Model(seed = idx, save_path = SAVE_PATH, epoch = 20, max_patience = 5)
  full_model.train()

  if full_model.max_val_acc >= threshold :
    max_idx -=1
    full_model.predict_soft()
    if full_pred_list is None :
      full_pred_list = full_model.get_predict_list()
    else :
      full_pred_list += full_model.get_predict_list()

  if not max_idx :
    break
  idx += 1

print(full_pred_list.shape)



Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnetv2_rw_m_agc-3d90cb1e.pth" to /root/.cache/torch/hub/checkpoints/efficientnetv2_rw_m_agc-3d90cb1e.pth
100%|██████████| 25/25 [00:03<00:00,  7.98it/s]


 [ epoch : 001 ] train_acc : 0.796, train_loss : 0.058, val_acc : 0.865, val_loss : 0.036 
Deteced New Best Acc, Model Save....


100%|██████████| 25/25 [00:03<00:00,  8.02it/s]


 [ epoch : 002 ] train_acc : 0.870, train_loss : 0.043, val_acc : 0.940, val_loss : 0.020 
Deteced New Best Acc, Model Save....


100%|██████████| 25/25 [00:03<00:00,  8.01it/s]


...

 [ epoch : 019 ] train_acc : 0.996, train_loss : 0.002, val_acc : 0.985, val_loss : 0.005 


100%|██████████| 25/25 [00:03<00:00,  7.50it/s]


 [ epoch : 020 ] train_acc : 0.994, train_loss : 0.002, val_acc : 0.985, val_loss : 0.004 


100%|██████████| 50/50 [00:06<00:00,  8.28it/s]

(400, 2)


In [8]:
print(full_pred_list[:5])

full_pred_list = np.argmax(full_pred_list, axis = 1)
full_pred_list = np.where(full_pred_list == 1, "normal", "covid")

SUBMISSION_PATH = os.path.join(DATA_PATH, "submission.csv")
submission_df = pd.read_csv(SUBMISSION_PATH)
submission_df['label']= full_pred_list
submission_df.to_csv("/kaggle/working/output_submission.csv",index=False)

[[ 32.788155 -34.65986 ]
 [ 19.186852 -19.254026]
 [-28.057928  29.373667]
 [ 30.672955 -31.777313]
 [-18.158539  19.40971 ]]
